# PYTORCH CNN Classifier

In [69]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [70]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_mouse_enhancers 64 subword 2


## Config

In [71]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [72]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [73]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [74]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=sample.csv --model_prefix=sample --vocab_size=64 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: sample.csv
  input_format: 
  model_prefix: sample
  model_type: UNIGRAM
  vocab_size: 64
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
no

vocab len: 64
{'ACC': 62, 'AATA': 61, 'TAC': 58, 'ACT': 57, 'GCA': 55, 'N': 54, 'AAAT': 53, 'AAGG': 52, 'AGC': 51, 'CAGA': 49, 'AAGA': 48, 'CAGC': 47, 'CAGT': 45, 'TCC': 44, 'AAAG': 19, 'AAC': 38, 'TGTT': 34, '<pad>': 63, 'C': 13, 'TTTT': 43, 'TCTG': 39, 'CTT': 33, 'TGC': 18, 'NNNNNNNNNNNNNNNN': 32, 'GAG': 30, 'TAT': 20, 'ATC': 37, 'AAAA': 59, 'TTA': 27, 'TCTA': 29, 'ATGA': 46, 'GAC': 15, 'CCC': 25, 'CCA': 40, 'CAA': 21, 'CAC': 22, '<unk>': 0, 'ATGT': 17, 'GAA': 4, 'NNNNNNNNNNNNNNN': 31, 'TGGC': 11, 'TTG': 28, 'CTGT': 60, 'AGTT': 7, 'G': 6, 'A': 2, 'GGA': 9, 'CAT': 5, 'CTGA': 24, 'GAT': 56, 'GGG': 3, 'GTA': 36, 'AGGC': 26, 'CCT': 8, 'GCT': 14, 'CTC': 42, 'TGGT': 12, '▁': 1, 'T': 10, 'TTC': 35, 'ATT': 41, 'GTG': 23, 'GCC': 50, 'GTC': 16}


## Dataloader and batch preparation

In [75]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device
max_tok_len  1738
not all sequences are of the same length


In [76]:
tokenizer(train_dset[1][0])

['▁',
 'AGC',
 'CCT',
 'CAA',
 'GGA',
 'GAC',
 'A',
 'CAGC',
 'A',
 'T',
 'A',
 'CTGT',
 'A',
 'CTT',
 'T',
 'CCA',
 'CAT',
 'A',
 'A',
 'TTG',
 'CTT',
 'TTC',
 'A',
 'G',
 'GTC',
 'TGC',
 'CTC',
 'CCA',
 'CAC',
 'A',
 'ATC',
 'CTT',
 'CAGT',
 'G',
 'AATA',
 'CCT',
 'A',
 'A',
 'GTC',
 'AGC',
 'AAAA',
 'CAA',
 'C',
 'A',
 'GGA',
 'A',
 'TTTT',
 'TAT',
 'ATT',
 'GAT',
 'CCT',
 'G',
 'AAAT',
 'ACT',
 'G',
 'G',
 'CAA',
 'G',
 'TCTG',
 'T',
 'A',
 'AGTT',
 'A',
 'T',
 'GGG',
 'ACT',
 'CAGC',
 'T',
 'GGG',
 'GCA',
 'AAAA',
 'CAA',
 'AATA',
 'TTG',
 'GCA',
 'ACT',
 'G',
 'CAT',
 'A',
 'GGA',
 'A',
 'CTT',
 'CTT',
 'T',
 'A',
 'A',
 'T',
 'AAGA',
 'CAGT',
 'GTA',
 'GAA',
 'AGC',
 'ATGT',
 'CCT',
 'TTTT',
 'C',
 'T',
 'TCC',
 'CCT',
 'ACC',
 'CAGA',
 'TGGC',
 'A',
 'G',
 'G',
 'T',
 'T',
 'TCTG',
 'TTTT',
 'CTT',
 'A',
 'CAC',
 'A',
 'A',
 'T',
 'GCC',
 'CAT',
 'TTG',
 'T',
 'A',
 'TGGC',
 'A',
 'ATT',
 'A',
 'TGC',
 'CCT',
 'A',
 'ACC',
 'G',
 'GCT',
 'T',
 'T',
 'CAGT',
 'GAA',
 'TGC',
 'AT

## Model

In [77]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [78]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 69.8%, Avg loss: 0.624183 

Epoch 1
Train metrics: 
 Accuracy: 79.3%, Avg loss: 0.593320 

Epoch 2
Train metrics: 
 Accuracy: 77.9%, Avg loss: 0.589330 

Epoch 3
Train metrics: 
 Accuracy: 79.3%, Avg loss: 0.591438 

Epoch 4
Train metrics: 
 Accuracy: 78.7%, Avg loss: 0.586046 



## Testing

In [79]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  121 ; tp  76.07450151443481 ; fp  13.56971947834063
recall  0.628714888549048 ; precision  0.8486269462988112
num_batches 8
correct 183
size 242
Test metrics: 
 Accuracy: 0.756198, F1 score: 0.722303, Avg loss: 0.602928 



(0.756198347107438, 0.7223032386636798)